In [ ]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster

# Carregar os dados (já classificados em quadrantes)
url = "Dataset_de_200_Comunidades_para_Analise_Ambiental2.csv"
data = pd.read_csv(url)

# Definir limiares e classificar (como no código anterior)
desmatamento_limite = 50
queimadas_limite = 5

def classificar_quadrante(row):
    if row['Índice de Desmatamento (%)'] >= desmatamento_limite and row['Frequência de Queimadas (ano)'] >= queimadas_limite:
        return '1-Alto Desmatamento + Alta Queimadas'
    elif row['Índice de Desmatamento (%)'] < desmatamento_limite and row['Frequência de Queimadas (ano)'] >= queimadas_limite:
        return '2-Baixo Desmatamento + Alta Queimadas'
    elif row['Índice de Desmatamento (%)'] >= desmatamento_limite and row['Frequência de Queimadas (ano)'] < queimadas_limite:
        return '3-Alto Desmatamento + Baixa Queimadas'
    else:
        return '4-Baixo Desmatamento + Baixa Queimadas'

data['Quadrante'] = data.apply(classificar_quadrante, axis=1)

# Criar mapa centrado no Pará
mapa = folium.Map(location=[-4.0, -52.0], zoom_start=6, tiles='CartoDB positron')

# Definir cores para cada quadrante
cores = {
    '1-Alto Desmatamento + Alta Queimadas': 'red',
    '2-Baixo Desmatamento + Alta Queimadas': 'orange',
    '3-Alto Desmatamento + Baixa Queimadas': 'purple',
    '4-Baixo Desmatamento + Baixa Queimadas': 'green'
}

# Adicionar marcadores agrupados para cada quadrante
for quadrante in data['Quadrante'].unique():
    # Filtrar dados para o quadrante atual
    df_quadrante = data[data['Quadrante'] == quadrante]
    
    # Criar cluster para o quadrante
    marker_cluster = MarkerCluster(
        name=quadrante,
        overlay=True,
        control=True,
        icon_create_function=None
    )
    
    # Adicionar marcadores ao cluster
    for idx, row in df_quadrante.iterrows():
        # Criar popup com informações
        popup_text = f"""
        <b>Comunidade:</b> {row['Comunidade']}<br>
        <b>Município:</b> {row['Município']}<br>
        <b>Desmatamento:</b> {row['Índice de Desmatamento (%)']}%<br>
        <b>Queimadas/ano:</b> {row['Frequência de Queimadas (ano)']}<br>
        <b>Cobertura Vegetal:</b> {row['Cobertura Vegetal (%)']}%<br>
        <b>Renda Média:</b> R$ {row['Renda Média Mensal (R$)']:,.2f}
        """
        popup = folium.Popup(popup_text, max_width=300)
        
        # Adicionar marcador ao cluster
        folium.Marker(
            location=[row['Latitude'], row['Longitude']],
            popup=popup,
            icon=folium.Icon(color=cores[quadrante], icon='info-sign'),
            tooltip=f"{row['Comunidade']} - {quadrante}"
        ).add_to(marker_cluster)
    
    # Adicionar cluster ao mapa
    marker_cluster.add_to(mapa)

# Adicionar controle de camadas
folium.LayerControl().add_to(mapa)

# Adicionar título ao mapa
title_html = '''
     <h3 align="center" style="font-size:16px"><b>Comunidades do Pará - Classificação por Desmatamento e Queimadas</b></h3>
     '''
mapa.get_root().html.add_child(folium.Element(title_html))

# Adicionar legenda
legend_html = '''
     <div style="position: fixed; 
                 bottom: 50px; left: 50px; width: 180px; height: 150px; 
                 border:2px solid grey; z-index:9999; font-size:12px;
                 background-color:white;
                 padding: 10px;
                 ">
       <b>Legenda:</b><br>
       <i style="background:red; width:15px; height:15px; display:inline-block;"></i> Alto D + Alta Q<br>
       <i style="background:orange; width:15px; height:15px; display:inline-block;"></i> Baixo D + Alta Q<br>
       <i style="background:purple; width:15px; height:15px; display:inline-block;"></i> Alto D + Baixa Q<br>
       <i style="background:green; width:15px; height:15px; display:inline-block;"></i> Baixo D + Baixa Q<br>
       <b>D:</b> Desmatamento<br>
       <b>Q:</b> Queimadas
     </div>
     '''
mapa.get_root().html.add_child(folium.Element(legend_html))

# Salvar mapa
#mapa.save('mapa_quadrantes_desmatamento_queimadas.html')

# Mostrar o mapa (em um ambiente que suporte exibição de HTML)
display(mapa)